# Save grain opacity (as a function of T, grain size, and wavelength)

In [1]:
from util import *
from interp_table import InterpTable
import dsharp_opac # this can be installed from https://github.com/birnstiel/dsharp_opac
import pickle

## Step 1. Compute opacity tables with the DSHARP package

To change resolution of the table (in grain size and wavelength) or cover different ranges of parameters, update the computation grid below.

In [7]:
# only set to true when computing grain properties for the first time
# or when you want to update grain properties (e.g., use a different grid)
compute_mie = False

# gird for computing grain properties
# grain size grid
a_min = 1e-7 # 0.1 um
a_max = 10 # 10 cm
N_a = 161 # 20 points per order of magnitude
a_grid = np.logspace(np.log10(a_min),np.log10(a_max),N_a)
# wavelength grid
lam_min = 1e-5 # 1000K=0.0002898cm, choose lam_min << this
lam_max = 10 # 10K = 0.02898cm, choose lam_max >> this and > max observed wavelength
N_lam = 121 # 20 points per order of magnitude
lam_grid = np.logspace(np.log10(lam_min),np.log10(lam_max),N_lam)

In [4]:
if compute_mie:
    # dust grain compoistion following Birnstiel et al. 2018
    # the four species:
    # water, scilicate, troilite, refractory organics
    N_composition = 4
    rho_grain = np.array([0.92, 3.30, 4.83, 1.50])
    mass_frac = np.array([0.2, 0.3291, 0.0743, 0.3966])
    vol_frac = np.array([0.3642, 0.1670, 0.0258, 0.4430])

    # sublimation temperature from Pollack et al. 1994
    T_crit = np.array([150, 425, 680, 1200])

    diel_constants = [dsharp_opac.diel_warrenbrandt08(),
                      dsharp_opac.diel_draine2003(species='astrosilicates'),
                      dsharp_opac.diel_henning('troilite'),
                      dsharp_opac.diel_henning('organics', refractory=True),
                     ]

    species_exists = [[1,1,1,1],
                      [0,1,1,1],
                      [0,1,1,0],
                      [0,1,0,0]]
    # species_exits[i,j] = species j exists in temperature range i
    species_exists = np.array(species_exists)
    rho_grain_eff = np.zeros(N_composition)
    mass_ratio_after_subl = np.ones(N_composition)
    mixed_diel_constants = [None]*N_composition
    for i in range(N_composition):
        mass_ratio_after_subl[i] = np.sum(mass_frac*species_exists[i])
        current_vol_frac = vol_frac*species_exists[i]
        current_vol_frac = current_vol_frac/np.sum(current_vol_frac)
        rho_grain_eff[i] = np.sum(current_vol_frac*rho_grain)
        mixed_diel_constants[i] = dsharp_opac.diel_mixed(constants=diel_constants,
                                  abundances=current_vol_frac,
                                  rule='Bruggeman')
        mixed_diel_constants[i] = mixed_diel_constants[i].get_normal_object()

    mie_data_package = [None]*N_composition
    for i in range(N_composition):
        mie_data_package[i] = dsharp_opac.get_mie_coefficients(
            a_grid, lam_grid, mixed_diel_constants[i],
            nang=3, extrapolate_large_grains=True) # nang follows the default value in dsharp_opac

    kappa   = [None]*N_composition # abroption opacity
    kappa_s = [None]*N_composition # scattering opacity
    g       = [None]*N_composition # asymmetry factor
    for i in range(N_composition):
        m = 4*np.pi/3 * a_grid**3 * rho_grain_eff[i]
        kappa_both = dsharp_opac.get_kappa_from_q(
            a_grid, m,
            mie_data_package[i]['q_abs'],
            mie_data_package[i]['q_sca'],
        )
        kappa[i] = kappa_both[0]
        kappa_s[i] = kappa_both[1]
        g[i] = mie_data_package[i]['g']

Please cite Warren & Brandt (2008) when using these optical constants
Please cite Draine 2003 when using these optical constants
Reading opacities from troilitek
Please cite Henning & Stognienko (1996) when using these optical constants
Reading opacities from organicsk
Please cite Henning & Stognienko (1996) when using these optical constants
Mie ... Done!
Mie ... Done!
Mie ... Done!
Mie ... Done!


In [5]:
if compute_mie:
    grain_properties = {}
    grain_properties['a_grid'] = a_grid
    grain_properties['lam_grid'] = lam_grid
    grain_properties['kappa'] = kappa
    grain_properties['kappa_s'] = kappa_s
    grain_properties['g'] = g
    grain_properties['T_crit'] = T_crit
    grain_properties['mass_ratio_after_subl'] = mass_ratio_after_subl
    pickle.dump(grain_properties, open('./data/opacity_tables/grain_properties.pkl', 'wb'))

In [6]:
grain_properties = pickle.load(open('./data/opacity_tables/grain_properties.pkl', 'rb'))
a_grid = grain_properties['a_grid']
lam_grid = grain_properties['lam_grid']
kappa = grain_properties['kappa']
kappa_s = grain_properties['kappa_s']
g = grain_properties['g']
T_crit = grain_properties['T_crit']
mass_ratio_after_subl = grain_properties['mass_ratio_after_subl']
N_composition = 4

## Setp 2. save results to an interpolatipon table

In [8]:
T_grid = np.sort(np.concatenate((T_crit-small_number, T_crit+small_number)))

grain_opacity = InterpTable()

grain_opacity.add_grid('log_T', np.log(T_grid))
grain_opacity.add_grid('log_a', np.log(a_grid))
grain_opacity.add_grid('log_lam', np.log(lam_grid))

grain_opacity.add_data('log_kappa_abs') # absorption opacity
grain_opacity.add_data('log_kappa_sca') # scattering opacity
grain_opacity.add_data('log_kappa_sca_eff') # effective scattering opacity
grain_opacity.add_data('g') # forward scattering paramater g (this can be negative)
grain_opacity.add_data('log_mass_fraction') # mass ratio after sublimation

for j in range(N_composition*2):
    i = (j+1)//2
    if i>=N_composition: i=N_composition-1
    grain_opacity.data['log_kappa_abs'][j] = np.log(kappa[i])
    grain_opacity.data['log_kappa_sca'][j] = np.log(kappa_s[i])
    grain_opacity.data['log_kappa_sca_eff'][j] = np.log(kappa_s[i]*(1-g[i]))
    grain_opacity.data['g'][j] = g[i]
    grain_opacity.data['log_mass_fraction'][j] = np.log(mass_ratio_after_subl[i])
grain_opacity.data['log_mass_fraction'][-1] = np.log(small_number)

grain_opacity.create_interp_fn_all_data()

print(grain_opacity)

InterpTable object with 3 dims
Axes:
  [0] log_T, length=8
  [1] log_a, length=161
  [2] log_lam, length=121
Data fields, shape=[8, 161, 121]:
  log_kappa_abs
  log_kappa_sca
  log_kappa_sca_eff
  g
  log_mass_fraction


In [10]:
update_grain_opacity = False
if update_grain_opacity:
    pickle.dump(grain_opacity, open('./data/opacity_tables/grain_opacity.pkl','wb'))